# Payoff Matrices (part 1)

> This module contains payoff matrices for different evolutionary games
>
> Part 1 contains payoff matrices for the following games
> - DSAIR
> - DSAIR with peer punishment or reward
> - DSAIR with voluntary commitments
>
> Note that all of the payoff matrices here are replications of the models from The Anh et al. 2020, 2021, 2022.

In [ ]:
#| default_exp payoffs

In [ ]:
#| hide
from nbdev.showdoc import *
from gh_pages_example.utils import *

In [ ]:
#| export

import nptyping
import numpy as np

#### DSAIR Model Paramaters

| keyword | value type | range | optional | description | 
|---------|------------|-------|----------|-------------|
| b | NDArray | b > 0| | The size of the per round benefit of leading the AI development race|
| c | NDArray | c > 0| | The cost of implementing safety recommendations per round|
| s | NDArray | s > 1| | The speed advantage from choosing to ignore safety recommendations|
| p | NDArray | [0, 1]| | The probability that unsafe firms avoid an AI disaster|
| B | NDArray | B >> b| | The size of the prize from winning the AI development race|
| W | NDArray | $$[10, 10^6]$$| | The anticipated timeline until the development race has a winner if everyone behaves safely|
| pfo | NDArray | [0, 1]|Yes| The probability that firms who ignore safety precautions are found out|
| ϵ | NDArray | ϵ > 0|Yes| The cost of setting up a voluntary commitment|
| ω | NDArray | [0, 1]|Yes| Noise in arranging an agreement, with some probability they fail to succeed in making an agreement|

#### DSAIR Payoff Matrix (Short Run)

| Strategy | Safe | Unsafe |
|----------|---|---|
| **Safe** | $$\frac{b}{2} - c$$|  $$\frac{b}{s+1} - c$$ |
| **Unsafe** | $$b \frac{s}{s+1}$$| $$\frac{b}{2} $$|

In [ ]:
#| export

def payoffs_sr(models):
    """The short run payoffs for the DSAIR game."""
    s, b, c = [models[k] for k in ['s', 'b', 'c']]
    πAA = -c + b/2
    πAB = -c + b/(s+1)
    πBA = s*b/(s+1)
    πBB = b/2
    matrix = np.block([[πAA, πAB], 
                       [πBA, πBB]])
    return {**models, 'payoffs_sr':matrix}

#### DSAIR Payoff Matrix (Short Run) with probability of being found out

| Strategy | Safe | Unsafe |
|----------|---|---|
| **Safe** | $$\frac{b}{2} - c$$|  $$(1 - p_{fo}) \frac{b}{s+1} + p_{fo} b - c$$ |
| **Unsafe** | $$ (1 - p_{fo}) b \frac{s}{s+1}$$| $$(1 - p_{fo}^2) \frac{b}{2} $$|

In [ ]:
#| export

def payoffs_sr_pfo_extension(model):
    """The short run payoffs for the DSAIR game with a chance of unsafe
    behaviour being spotted."""
    s, b, c, pfo = [model[k] for k in ['s', 'b', 'c', 'pfo']]
    πAA =  -c + b/2
    πAB = -c + b/(s+1) * (1 - pfo) + pfo * b
    πBA = (1 - pfo) * s * b / (s+1)
    πBB = (1 - pfo**2) * b/2
    matrix = np.block([[πAA, πAB],
                       [πBA, πBB]])
    return {**model, 'payoffs_sr':matrix}

#### DSAIR Payoff Matrix (Long Run)

Denote $\pi$ as one of the short run payoff matrices discussed above with rows and columns indexed by letters A, B, ...

| Strategy | Always Safe | Always Unsafe |
|----------|---|---|
| **Always Safe** | $$πAA + \frac{B}{2W}$$|  $$πAB$$ |
| **Always Unsafe** | $$p \, (s \frac{B}{W} + πBA)$$| $$p \, (s \frac{B}{2W} + πBB)$$|

*Note: In a model where we suffer a collective risk of an AI disaster if the winner is unsafe, payoffs for firms who play safe when facing an unsafe firm are also multiplied by $p$.*

In [ ]:
#| export

def payoffs_lr(model):
    """The long run average payoffs for the DSAIR game."""
    s, p, B, W = [model[k] for k in ['s', 'p', 'B', 'W']]
    πAA,πAB,πBA,πBB = [model['payoffs_sr'][:,[i],[j]]
                       for i in range(2) for j in range(2)]
    πAA = πAA + B/(2*W)
    πAB = πAB
    πBA = p*(s*B/W + πBA)
    πBB = p*(s*B/(2*W) + πBB)
    payoffs = np.block([[πAA, πAB],
                        [πBA, πBB]])
    return {**model, 'payoffs': payoffs}

#### DSAIR Payoff Matrix with punishments (Long Run)

Denote $\pi$ as one of the short run payoff matrices discussed above with rows and columns indexed by letters A, B, ...

**Always Safe** and **Always Unsafe** play as they usually do.

**Punish Unsafe** always plays Safe. However, they will pay a cost to punish their co-player if the co-player plays Unsafe.


| Strategy | Always Safe | Always Unsafe | Punish Unsafe |
|----------|---|---|---|
| **Always Safe** | $$πAA + \frac{B}{2W}$$|  $$πAB$$ | $$πAA + \frac{B}{2W}$$ |
| **Always Unsafe** | $$p \, (s \frac{B}{W} + πBA)$$| $$p \, (s \frac{B}{2W} + πBB)$$| punished_payoff|
| **Punish Unsafe** | $$πAA + \frac{B}{2W}$$| sanctioner_payoff | $$πAA + \frac{B}{2W}$$ |

In [ ]:
#| export

def payoffs_lr_peer_punishment(model):
    """The long run average payoffs for the DSAIR game with peer punishment."""
    s,b,c, p, B, W = [model[k] for k in ['s', 'b', 'c', 'p', 'B', 'W']]
    α, γ, ϵ = [model[k] for k in ['α', 'γ', 'ϵ']]
    πAA,πAB,πBA,πBB = [model['payoffs_sr'][:,[i],[j]]
                       for i in range(2) for j in range(2)]
    
    s_punished = s - γ
    s_sanctioner = s - α
    punished_wins = (s > γ) & ((W-s)/(s-γ) <= (W-1)/(s-α))
    punished_draws = (s > γ) & ((W-s)/(s-γ) == (W-1)/(s-α))
    sanctioner_wins = (s > α) & ((W-s)/(s-γ) >= (W-1)/(s-α))
    no_winner = (s_punished <= 0) & (s_sanctioner <= 0)

    both_speeds_positive = (s_punished > 0) & (s_sanctioner > 0)
    only_sanctioner_speed_positive = (s_punished <= 0) & (s_sanctioner > 0)
    only_punisher_speed_positive = (s_punished > 0) & (s_sanctioner <= 0)

    p_loss = np.where(punished_wins | punished_draws, p, 1)
    R = np.where(no_winner,
                 np.inf,
                 1 + np.minimum((W-s)/ np.maximum(s_punished, 1e-10),
                                (W-1)/ np.maximum(s_sanctioner, 1e-10)))
    B_s = np.where(sanctioner_wins, B, np.where(punished_draws, B/2, 0))
    B_p = np.where(punished_wins, B, np.where(punished_draws, B/2, 0))
    b_s = np.where(both_speeds_positive,
                   b * (s - α)/(s-γ + s - α),
                   np.where(only_sanctioner_speed_positive, b, 0))
    b_p = np.where(both_speeds_positive,
                   b * (s - γ)/(s-γ + s - α),
                   np.where(only_punisher_speed_positive, b, 0))

    sanctioner_payoff = (1 / R) * (πAB + B_s + (R-1)*(b_s - c)) - ϵ
    punished_payoff = (p_loss / R) * (πBA + B_p + (R-1)*b_p) - ϵ
    
    ΠAA = πAA + B/(2*W)
    ΠAB = p*(s*B/W + πBA)
    ΠAC = πAA + B/(2*W)
    ΠBA = p*(s*B/W + πBA)
    ΠBB = p*(s*B/(2*W) + πBB)
    ΠBC = punished_payoff
    ΠCA = πAA + B/(2*W)
    ΠCB = sanctioner_payoff
    ΠCC = πAA + B/(2*W)
    matrix = np.block([[ΠAA, ΠAB, ΠAC], 
                       [ΠBA, ΠBB, ΠBC],
                       [ΠCA, ΠCB, ΠCC],
                       ])
    return {**model, 'payoffs':matrix}

##### Expressions for the sanctioner and punished payoffs

For convenience we denote a number of new variables to simplify the expressions for the sanctioner and punished payoffs.

\begin{equation}
\text{sanctioner payoff} = \frac{1}{R} (\pi AB + B_s + (R-1) (b_s - c))\\
\end{equation}

\begin{equation}
\text{punished payoff} = \frac{p_{punish}}{R} (πBA + B_p + (R-1) b_p)\\
\end{equation}

*Note: In a model where we suffer a collective risk of an AI disaster if the winner is unsafe, payoffs for firms who play safe when facing an unsafe firm are also multiplied by $p_{punish}$.*

We can read the above payoffs as telling us the average payoffs over the R rounds of the race for each firm, assuming the punishment is levied at the end of the first round and the remaining $R - 1$ rounds are played with the punishment in effect.

Note that $s_{\beta}$ denotes the new speed of the firm who is punished and $s_{\alpha}$ as the speed of the firm who levies the punishment.

Below we denote the four possible outcomes (ignoring disaster) of a race between a sanctioner and a punished firm:

\begin{equation}
\text{punished wins} = (s_{\beta} > 0) \, \& \, (\frac{W-s}{s_{\beta}} <= \frac{W-1}{s_{\alpha}})
\end{equation}

\begin{equation}
\text{sanctioner wins} = (s_{\alpha} > 0) \, \& \, (\frac{W-1}{s_{\alpha}} <= \frac{W-s}{s_{\beta}})
\end{equation}

\begin{equation}
\text{draw} = (s_{\beta} > 0) \, \& \, (\frac{W-s}{s_{\beta}} = \frac{W-1}{s_{\alpha}})
\end{equation}

\begin{equation}
\text{no winner} = (s_{\beta} <= 0)  \, \& \,  (s_{\alpha} <= 0)
\end{equation}

We can use the above expressions to define the following variables:

$p_{punish}$ is the probability of avoiding an AI disaster if a punishment is levied and depends on who wins the race.

\begin{equation}
p_{punish} = \begin{cases} 0 & \text{sanctioner wins | no winner} \\
p & otherwise
\end{cases}
\end{equation}

$R$ is the number of rounds that the race lasts for; the race ends when the first firm reaches the finish line.

\begin{equation}
R = \begin{cases} \infty & \text{no winner} \\
\frac{W - 1}{s_{\alpha}} & \text{sanctioner wins} \\
\frac{W - s}{s_{\beta}} & \text{punished wins | draw} \\
\end{cases}
\end{equation}

$B_s$ is the prize that the sanctioner receives at the end of the race.

\begin{equation}
B_s = \begin{cases} B & \text{sanctioner wins} \\
\frac{B}{2} & \text{draw} \\
0 & otherwise \\
\end{cases}
\end{equation}

$B_p$ is the prize that the punished receives at the end of the race.

\begin{equation}
B_p = \begin{cases} B & \text{punished wins} \\
\frac{B}{2} & \text{draw} \\
0 & otherwise \\
\end{cases}
\end{equation}

$b_s$ is the benefit the sanctioner receives each round, they only gain a benefit if their speed is positive but gain the whole benefit if they are the only firm with positive speed.

\begin{equation}
b_s = \begin{cases} p_{fo} b + (1-p_{fo}) b \frac{s_{\alpha}}{s_{\alpha} + s_{\beta}} & s_{\alpha}, s_{\beta} > 0\\
b & s_{\alpha} > 0 >= s_{\beta} \\
0 & s_{\alpha} <= 0 \\
\end{cases}
\end{equation}

$b_p$ is the benefit the punished receives each round, they only gain a benefit if their speed is positive but gain the whole benefit if they are the only firm with positive speed.

\begin{equation}
b_p = \begin{cases} (1-p_{fo}) b \frac{s_{\beta}}{s_{\alpha} + s_{\beta}} & s_{\alpha}, s_{\beta} > 0\\
b & s_{\beta} > 0 >= s_{\alpha} \\
0 & s_{\beta} <= 0 \\
\end{cases}
\end{equation}

#### DSAIR Payoff Matrix with rewards (Long Run)

Denote $\pi$ as one of the short run payoff matrices discussed above with rows and columns indexed by letters A, B, ...

**Always Safe** and **Always Unsafe** play as they usually do.

**Reward Safe** always plays Safe. However, they will pay a cost to reward their co-player if the co-player plays Safe.

| Strategy | Always Safe | Always Unsafe | Reward Safe |
|----------|---|---|---|
| **Always Safe** | $$πAA + \frac{B}{2W}$$|  $$πAB$$ | $$πAA + \frac{B (1 + s_{\beta})}{W}$$ |
| **Always Unsafe** | $$p \, (s \frac{B}{W} + πBA)$$| $$p \, (s \frac{B}{2W} + πBB)$$| $$p \, (s \frac{B}{W} + πBA)$$|
| **Reward Safe** | $$ πAA $$| $$ πAB $$| $$πAA + \frac{B (1 + s_{\beta} - s_{\alpha})}{2W}$$ |

In [ ]:
#| export

def payoffs_lr_peer_reward(model):
    """The long run average payoffs for the DSAIR game with peer punishment."""
    s,b,c, p, B, W = [model[k] for k in ['s', 'b', 'c', 'p', 'B', 'W']]
    α, γ, ϵ = [model[k] for k in ['α', 'γ', 'ϵ']]
    πAA,πAB,πBA,πBB = [model['payoffs_sr'][:,[i],[j]]
                       for i in range(2) for j in range(2)]
    
    s_rewarded = s + γ
    s_helper = s - α
    ΠAA = πAA + B/(2*W)
    ΠAB = p*(s*B/W + πBA)
    ΠAC = πAA + B * s_rewarded / ((s_rewarded + s_helper) * W)
    ΠBA = p*(s*B/W + πBA)
    ΠBB = p*(s*B/(2*W) + πBB)
    ΠBC = p*(s*B/W + πBA)
    ΠCA = πAA
    ΠCB = πAB
    ΠCC = πAA + B * (1 + s_rewarded - s_helper)/(2*W)
    matrix = np.block([[ΠAA, ΠAB, ΠAC], 
                       [ΠBA, ΠBB, ΠBC],
                       [ΠCA, ΠCB, ΠCC],
                       ])
    return {**model, 'payoffs':matrix}

#### DSAIR Payoff Matrix with voluntary commitments (Long Run)

Denote $\pi$ as one of the short run payoff matrices discussed above with rows and columns indexed by letters A, B, ...

The strategies below are less obvious than in earlier models. **Always Safe Out** and **Always Unsafe Out** are the same strategies we are used to.

**Always Safe In** is willing to form a commitment to play Safe. Otherwise, they will always play Unsafe.

**Always Unsafe In** is willing to form a commitment but will violate it by always playing Unsafe. This way, they anticipate that they can encourage other firms to play safe and so pull ahead of them in the race.

**Punish Violator** is willing to form a commitment to play Safe. Otherwise, they will always play Unsafe. If the coparty to the commitment violates the commitment by playing Unsafe, then this player pays a cost to levy a punishment on the violator.

| Strategy| Always Safe Out | Always Unsafe Out |  Always Safe In | Always Unsafe In  | Punish Violator |
|----------|---|---|---|---|---|
| **Always Safe Out** | $$πAA + \frac{B}{2W}$$| $$πAB$$ | $$πAB$$ | $$πAB$$ | $$πAB$$|
| **Always Unsafe Out** | $$p \, (s \frac{B}{W} + πBA)$$| $$p \, (s \frac{B}{2W} + πBB)$$| $$p \, (s \frac{B}{2W} + πBB)$$| $$p \, (s \frac{B}{2W} + πBB)$$ | $$p \, (s \frac{B}{2W} + πBB)$$ |
| **Always Safe In** | $$p \, (s \frac{B}{W} + πBA)$$|  $$p \, (s \frac{B}{2W} + πBB)$$ | $$πAA + \frac{B}{2W} - \epsilon$$| $$πAB - \epsilon$$| $$πAA + \frac{B}{2W} - \epsilon$$ |
| **Always Unsafe In** | $$p \, (s \frac{B}{W} + πBA)$$| $$p \, (s \frac{B}{2W} + πBB)$$| $$p \, (s \frac{B}{W} + πBA) - \epsilon$$| $$p \, (s \frac{B}{2W} + πBB) - \epsilon$$ | punished_payoff - ϵ |
| **Punish Violator** | $$p \, (s \frac{B}{W} + πBA)$$| $$p \, (s \frac{B}{2W} + πBB)$$| $$πAA + \frac{B}{2W} - \epsilon$$ | sanctioner_payoff - ϵ| $$πAA + \frac{B}{2W} - \epsilon$$ |

The punished and sanctioner payoffs above are exactly the same as in the model with punishments above, so I do not repeat this here.

In [ ]:
#| export

def payoffs_lr_voluntary(model):
    """The long run average payoffs for the DSAIR game with voluntary
    commitments."""
    s,b,c, p, B, W = [model[k] for k in ['s', 'b', 'c', 'p', 'B', 'W']]
    α, γ, ϵ = [model[k] for k in ['α', 'γ', 'ϵ']]
    πAA,πAB,πBA,πBB = [model['payoffs_sr'][:,[i],[j]]
                       for i in range(2) for j in range(2)]
    
    s_punished = s - γ
    s_sanctioner = s - α
    punished_wins = (s > γ) & ((W-s)/(s-γ) <= (W-1)/(s-α))
    punished_draws = (s > γ) & ((W-s)/(s-γ) == (W-1)/(s-α))
    sanctioner_wins = (s > α) & ((W-s)/(s-γ) >= (W-1)/(s-α))
    no_winner = (s_punished <= 0) & (s_sanctioner <= 0)

    both_speeds_positive = (s_punished > 0) & (s_sanctioner > 0)
    only_sanctioner_speed_positive = (s_punished <= 0) & (s_sanctioner > 0)
    only_punisher_speed_positive = (s_punished > 0) & (s_sanctioner <= 0)

    p_punish = np.where(punished_wins | punished_draws, p, 1)
    R = np.where(no_winner,
                 np.inf,
                 1 + np.minimum((W-s)/ np.maximum(s_punished, 1e-10),
                                (W-1)/ np.maximum(s_sanctioner, 1e-10)))
    B_s = np.where(sanctioner_wins, B, np.where(punished_draws, B/2, 0))
    B_p = np.where(punished_wins, B, np.where(punished_draws, B/2, 0))
    b_s = np.where(both_speeds_positive,
                   b * (s - α)/(s-γ + s - α),
                   np.where(only_sanctioner_speed_positive, b, 0))
    b_p = np.where(both_speeds_positive,
                   b * (s - γ)/(s-γ + s - α),
                   np.where(only_punisher_speed_positive, b, 0))

    sanctioner_payoff = (1 / R) * (πAB + B_s + (R-1)*(b_s - c)) - ϵ
    punished_payoff = (p_punish / R) * (πBA + B_p + (R-1)*b_p) - ϵ
    
    ΠAA = πAA + B/(2*W)
    ΠAB = πAB
    ΠAC = πAB
    ΠAD = πAB
    ΠAE = πAB
    ΠBA = p*(s*B/W + πBA)
    ΠBB = p*(s*B/(2*W) + πBB)
    ΠBC = p*(s*B/(2*W) + πBB)
    ΠBD = p*(s*B/(2*W) + πBB)
    ΠBE = p*(s*B/(2*W) + πBB)
    ΠCA = p*(s*B/W + πBA)
    ΠCB = p*(s*B/(2*W) + πBB)
    ΠCC = πAA + B/(2*W) - ϵ
    ΠCD = πAB - ϵ
    ΠCE = πAA + B/(2*W) - ϵ
    ΠDA = p*(s*B/W + πBA)
    ΠDB = p*(s*B/(2*W) + πBB)
    ΠDC = p*(s*B/W + πBA) - ϵ
    ΠDD = p*(s*B/(2*W) + πBB) - ϵ
    ΠDE = punished_payoff - ϵ
    ΠEA = p*(s*B/W + πBA) - ϵ
    ΠEB = p*(s*B/(2*W) + πBB)
    ΠEC = πAA + B/(2*W) - ϵ
    ΠED = sanctioner_payoff - ϵ
    ΠEE = πAA + B/(2*W) - ϵ
    matrix = np.block([[ΠAA, ΠAB, ΠAC, ΠAD, ΠAE], 
                       [ΠBA, ΠBB, ΠBC, ΠBD, ΠBE],
                       [ΠCA, ΠCB, ΠCC, ΠCD, ΠCE],
                       [ΠDA, ΠDB, ΠDC, ΠDD, ΠDE],
                       [ΠEA, ΠEB, ΠEC, ΠED, ΠEE]
                       ])
    return {**model, 'payoffs':matrix}

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()